In [2]:
%run "contant.py"

已加载全局配置

🎯 模型常量:
  OPENAI_MODEL, open_model

🎨 打印方法:
  pretty_print, print_json
  print_agent


## 工具调用（Tool Calling）

In [10]:
from typing import Annotated, TypedDict, Sequence
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain.tools import tool
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages


# ============ 1. 定义共享状态 ============
class ResearchState(TypedDict):
    """多 agent 之间共享的状态"""
    messages: Annotated[Sequence[BaseMessage], add_messages]
    research_topic: str
    search_results: str
    analysis_results: str
    final_report: str


# ============ 2. 定义 Agent 工具 ============

# 搜索 Agent 的工具
@tool
def search_papers(query: str) -> str:
    """搜索学术论文和资源。
    
    Args:
        query: 搜索关键词
    """
    # 模拟搜索结果
    results = [
        f"论文1: {query}的核心研究 - 2024年发表",
        f"论文2: {query}的最新进展 - 2023年发表",
        f"综述: {query}的全面分析 - 2023年发表",
    ]
    return "\n".join(results)


@tool
def get_paper_details(paper_id: str) -> str:
    """获取论文详细信息。
    
    Args:
        paper_id: 论文ID
    """
    return f"论文 {paper_id} 的详细信息：摘要、方法论、关键发现..."


# 分析 Agent 的工具
@tool
def analyze_data(data: str) -> str:
    """分析研究数据。
    
    Args:
        data: 需要分析的数据
    """
    return f"对数据的深度分析：\n- 关键发现：数据显示显著趋势\n- 统计意义：p < 0.05\n- 影响：{data}在领域中具有重要意义"


@tool
def compare_studies(study1: str, study2: str) -> str:
    """比较两项研究。
    
    Args:
        study1: 第一项研究
        study2: 第二项研究
    """
    return f"比较 {study1} 和 {study2}：\n- 方法差异\n- 结果对比\n- 结论的一致性"


# 总结 Agent 的工具
@tool
def generate_report_section(section_type: str, content: str) -> str:
    """生成报告的某个章节。
    
    Args:
        section_type: 章节类型 (摘要/背景/发现/结论)
        content: 章节内容
    """
    return f"生成的 {section_type} 章节：{content}"


@tool
def format_references(papers: str) -> str:
    """格式化参考文献。
    
    Args:
        papers: 论文列表
    """
    return f"格式化的参考文献：\n[1] {papers}\n[2] {papers}"


# ============ 3. 创建专业 Agent ============

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 搜索 Agent
search_agent = create_agent(
    model=model,
    tools=[search_papers, get_paper_details],
    system_prompt="你是一个学术研究搜索专家。你的任务是找到相关的学术资源和论文。"
)

# 分析 Agent
analysis_agent = create_agent(
    model=model,
    tools=[analyze_data, compare_studies],
    system_prompt="你是一个数据分析专家。你的任务是深度分析研究数据并识别关键模式。"
)

# 总结 Agent
summary_agent = create_agent(
    model=model,
    tools=[generate_report_section, format_references],
    system_prompt="你是一个技术写作专家。你的任务是将分析结果转化为清晰的报告。"
)


# ============ 4. 创建主管 Agent ============

@tool
def delegate_to_search_agent(task: str) -> str:
    """委托搜索 agent 执行搜索任务。"""
    result = search_agent.invoke({
        "messages": [HumanMessage(content=task)]
    })
    return result["messages"][-1].content


@tool
def delegate_to_analysis_agent(task: str) -> str:
    """委托分析 agent 执行分析任务。"""
    result = analysis_agent.invoke({
        "messages": [HumanMessage(content=task)]
    })
    return result["messages"][-1].content


@tool
def delegate_to_summary_agent(task: str) -> str:
    """委托总结 agent 执行总结任务。"""
    result = summary_agent.invoke({
        "messages": [HumanMessage(content=task)]
    })
    return result["messages"][-1].content


supervisor_agent = create_agent(
    model=model,
    tools=[
        delegate_to_search_agent,
        delegate_to_analysis_agent,
        delegate_to_summary_agent,
    ],
    system_prompt="""你是一个研究项目主管。你需要协调三个专家团队：
    1. 搜索团队：负责找到相关学术资源
    2. 分析团队：负责深度分析数据
    3. 总结团队：负责生成最终报告
    
    根据用户的研究主题，按照顺序委派任务：
    1. 先让搜索团队找论文
    2. 再让分析团队分析发现
    3. 最后让总结团队生成报告"""
)


# ============ 5. 构建 LangGraph 工作流 ============

def research_workflow():
    """创建多 agent 协同的 LangGraph 工作流"""
    
    workflow = StateGraph(ResearchState)
    
    # 定义节点
    def supervisor_node(state: ResearchState):
        """主管 agent 节点"""
        user_query = f"请为以下研究主题进行完整研究：{state['research_topic']}"
        
        result = supervisor_agent.invoke({
            "messages": state["messages"] + [HumanMessage(content=user_query)]
        })
        
        return {
            "messages": state["messages"] + result["messages"],
            "final_report": result["messages"][-1].content if result["messages"] else ""
        }
    
    # 添加节点和边
    workflow.add_node("supervisor", supervisor_node)
    workflow.add_edge(START, "supervisor")
    workflow.add_edge("supervisor", END)
    
    return workflow.compile()


# ============ 6. 运行完整示例 ============

def run_research_example():
    """运行完整的多 agent 研究流程"""
    
    graph = research_workflow()
    
    # 初始化状态
    initial_state = {
        "messages": [],
        "research_topic": "机器学习在医疗诊断中的应用",
        "search_results": "",
        "analysis_results": "",
        "final_report": ""
    }
    
    # 执行工作流
    print("=" * 60)
    print("开始多 Agent 协同研究...")
    print("=" * 60)
    
    final_state = graph.invoke(initial_state)
    
    print("\n" + "=" * 60)
    print("研究完成！")
    print("=" * 60)
    print("\n最终报告:")
    print(final_state["final_report"])


# ============ 7. 流式执行（实时查看每个 agent 的工作） ============

def run_research_with_streaming():
    """流式执行，实时查看每个 agent 的工作"""
    
    graph = research_workflow()
    
    initial_state = {
        "messages": [],
        "research_topic": "区块链技术在供应链管理中的应用",
        "search_results": "",
        "analysis_results": "",
        "final_report": ""
    }
    
    print("=" * 60)
    print("开始流式多 Agent 研究（实时输出）...")
    print("=" * 60)
    
    # 流式获取每一步的输出
    for step in graph.stream(initial_state, stream_mode="updates"):
        for node, output in step.items():
            print(f"\n[{node}] 节点执行中...")
            if "final_report" in output and output["final_report"]:
                print(f"报告内容:\n{output['final_report'][:200]}...")
    
    print("\n" + "=" * 60)
    print("流式执行完成！")
    print("=" * 60)


if __name__ == "__main__":
    # 运行示例
    run_research_example()
    
    # 或运行流式版本
    # run_research_with_streaming()

开始多 Agent 协同研究...

研究完成！

最终报告:
# 机器学习在医疗诊断中的应用报告

## 背景
机器学习（ML）作为人工智能的一个分支，近年来在医疗领域得到了广泛应用。随着医疗数据的快速增长，传统的诊断方法面临着数据处理能力不足的问题。机器学习通过分析大量的医疗数据，能够识别出潜在的模式和趋势，从而辅助医生进行更准确的诊断。特别是在影像学、基因组学和电子健康记录等领域，机器学习技术展现出了巨大的潜力。

## 关键发现
研究表明，机器学习在医疗诊断中的应用能够显著提高诊断的准确性和效率。例如，在影像学中，深度学习算法能够自动识别肿瘤和其他病变，减少了人工分析的时间和误差。此外，机器学习还可以通过分析患者的历史数据，预测疾病的发生风险，从而实现早期干预。许多临床试验和实际应用案例显示，机器学习模型在某些疾病的诊断上优于传统方法。

## 趋势
随着技术的进步，机器学习在医疗诊断中的应用趋势日益明显。首先，越来越多的医疗机构开始采用基于机器学习的决策支持系统，以提高诊断效率。其次，跨学科的合作日益增多，数据科学家与医疗专业人员的合作使得机器学习模型的开发更加贴近临床需求。此外，随着可穿戴设备和移动健康应用的普及，实时数据的收集和分析为机器学习在医疗诊断中的应用提供了新的机会。

## 挑战
尽管机器学习在医疗诊断中展现出巨大的潜力，但仍面临诸多挑战。首先，数据隐私和安全性问题是一个重要的障碍，医疗数据的敏感性要求在使用机器学习时必须遵循严格的伦理标准。其次，机器学习模型的可解释性不足，使得医生在使用这些工具时可能缺乏信心。此外，数据的质量和多样性也影响着模型的性能，如何获取高质量的标注数据仍然是一个亟待解决的问题。

---

如果您需要进一步的信息或有其他问题，请随时告诉我！


In [13]:
from typing import Annotated, Sequence, TypedDict, Literal
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.types import Command
from langgraph.checkpoint.memory import MemorySaver


# ============ 定义共享状态 ============
class SupportState(TypedDict):
    """多 agent 之间共享的对话状态"""
    messages: Annotated[Sequence[BaseMessage], add_messages]
    current_agent: str  # 当前活跃的 agent
    issue_complexity: Literal["simple", "medium", "complex"]  # 问题复杂度
    attempt_count: int  # 当前 agent 的尝试次数


# ============ 创建 LLM ============
model = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)


# ============ 定义一级支持 Agent ============
def tier1_support_node(state: SupportState):
    """一级技术支持 agent"""
    
    system_prompt = """你是一个友好的一级技术支持代表。
你的职责是：
1. 帮助用户解决简单的技术问题
2. 理解问题的复杂度
3. 如果你无法解决，明确告诉用户将转移给高级技术团队

如果问题超出你的能力范围，在回答末尾添加：[ESCALATE_TO_TIER2]"""
    
    messages = state["messages"]
    
    response = model.invoke(
        [{"role": "system", "content": system_prompt}] + 
        [{"role": msg.type, "content": msg.content} for msg in messages]
    )
    
    assistant_message = AIMessage(content=response.content)
    updated_messages = messages + [assistant_message]
    
    # 检查是否需要升级
    if "[ESCALATE_TO_TIER2]" in response.content:
        clean_content = response.content.replace("\n[ESCALATE_TO_TIER2]", "")
        updated_messages[-1] = AIMessage(content=clean_content)
        
        return Command(
            update={
                "messages": updated_messages,
                "current_agent": "tier1",
                "issue_complexity": "simple",
                "attempt_count": state.get("attempt_count", 0) + 1
            },
            goto="tier2_support"  # 转移给二级
        )
    
    return {
        "messages": updated_messages,
        "current_agent": "tier1",
        "issue_complexity": "simple",
        "attempt_count": state.get("attempt_count", 0) + 1
    }


# ============ 定义二级技术专家 Agent ============
def tier2_support_node(state: SupportState):
    """二级技术专家 agent"""
    
    system_prompt = """你是一个经验丰富的二级技术专家。
你的职责是：
1. 处理一级支持无法解决的问题
2. 进行深入的技术分析和调试
3. 如果涉及产品设计缺陷或需要产品更改，转移给产品经理

如果问题需要产品改进，在末尾添加：[ESCALATE_TO_PM]"""
    
    messages = state["messages"]
    
    escalation_context = "\n\n[背景] 一级支持无法解决此问题，已升级给您处理。"
    
    response = model.invoke(
        [{"role": "system", "content": system_prompt + escalation_context}] + 
        [{"role": msg.type, "content": msg.content} for msg in messages]
    )
    
    assistant_message = AIMessage(content=response.content)
    updated_messages = messages + [assistant_message]
    
    # 检查是否需要继续升级
    if "[ESCALATE_TO_PM]" in response.content:
        clean_content = response.content.replace("\n[ESCALATE_TO_PM]", "")
        updated_messages[-1] = AIMessage(content=clean_content)
        
        return Command(
            update={
                "messages": updated_messages,
                "current_agent": "tier2",
                "issue_complexity": "medium",
                "attempt_count": state.get("attempt_count", 0) + 1
            },
            goto="product_manager"  # 转移给产品经理
        )
    
    return {
        "messages": updated_messages,
        "current_agent": "tier2",
        "issue_complexity": "medium",
        "attempt_count": state.get("attempt_count", 0) + 1
    }


# ============ 定义产品经理 Agent ============
def product_manager_node(state: SupportState):
    """产品经理 agent"""
    
    system_prompt = """你是产品经理，负责处理最复杂的用户问题。
你的职责是：
1. 评估问题是否反映了产品设计的不足
2. 做出关于功能改进的决策
3. 提供解决方案或功能承诺"""
    
    messages = state["messages"]
    
    escalation_context = "\n\n[背景] 这是一个已经过二级技术专家评估的复杂问题，需要产品层面的决策。"
    
    response = model.invoke(
        [{"role": "system", "content": system_prompt + escalation_context}] + 
        [{"role": msg.type, "content": msg.content} for msg in messages]
    )
    
    assistant_message = AIMessage(content=response.content)
    updated_messages = messages + [assistant_message]
    
    # 产品经理是最后一级，问题已解决
    return {
        "messages": updated_messages,
        "current_agent": "product_manager",
        "issue_complexity": "complex",
        "attempt_count": state.get("attempt_count", 0) + 1
    }


# ============ 构建 LangGraph 工作流 ============
def create_support_workflow():
    """创建握手转移的多层级支持系统"""
    
    workflow = StateGraph(SupportState)
    
    # 添加节点
    workflow.add_node("tier1_support", tier1_support_node)
    workflow.add_node("tier2_support", tier2_support_node)
    workflow.add_node("product_manager", product_manager_node)
    
    # 设置起点：从一级支持开始
    workflow.add_edge(START, "tier1_support")
    
    # 默认结束
    workflow.add_edge("tier1_support", END)
    workflow.add_edge("tier2_support", END)
    workflow.add_edge("product_manager", END)
    
    # 编译（这里添加 checkpointer）
    checkpointer = MemorySaver()
    return workflow.compile(checkpointer=checkpointer)


# ============ 修复：提供 config 参数 ============

def test_simple_issue():
    """测试简单问题：密码重置"""
    print("\n" + "="*60)
    print("场景 1：简单问题 - 密码重置")
    print("="*60)
    
    graph = create_support_workflow()
    
    initial_state = {
        "messages": [
            HumanMessage(content="我忘记了账户密码，该怎么办？")
        ],
        "current_agent": "tier1",
        "issue_complexity": "simple",
        "attempt_count": 0
    }
    
    # 关键修复：提供 config 参数，包含 thread_id
    config = {"configurable": {"thread_id": "support_001"}}
    
    final_state = graph.invoke(initial_state, config)
    
    print("\n对话历史:")
    for msg in final_state["messages"]:
        role = "用户" if msg.type == "human" else "支持"
        print(f"\n[{role}]:")
        print(msg.content)
    
    print(f"\n最终处理 agent: {final_state['current_agent']}")


def test_medium_issue():
    """测试中等问题：性能优化"""
    print("\n" + "="*60)
    print("场景 2：中等问题 - 应用性能缓慢")
    print("="*60)
    
    graph = create_support_workflow()
    
    initial_state = {
        "messages": [
            HumanMessage(content="我们的应用在处理 100 万用户并发时性能严重下降。这是应用架构的问题吗？")
        ],
        "current_agent": "tier1",
        "issue_complexity": "medium",
        "attempt_count": 0
    }
    
    # 提供 config 参数
    config = {"configurable": {"thread_id": "support_002"}}
    
    final_state = graph.invoke(initial_state, config)
    
    print("\n对话历史:")
    for msg in final_state["messages"]:
        role = "用户" if msg.type == "human" else "支持"
        print(f"\n[{role}]:")
        content = msg.content[:300] + "..." if len(msg.content) > 300 else msg.content
        print(content)
    
    print(f"\n最终处理 agent: {final_state['current_agent']}")


def test_complex_issue():
    """测试复杂问题：产品设计缺陷"""
    print("\n" + "="*60)
    print("场景 3：复杂问题 - 产品设计缺陷")
    print("="*60)
    
    graph = create_support_workflow()
    
    initial_state = {
        "messages": [
            HumanMessage(content="我们发现当用户同时使用多个设备时，实时同步会产生数据冲突。这需要怎样的产品层面的改进？")
        ],
        "current_agent": "tier1",
        "issue_complexity": "complex",
        "attempt_count": 0
    }
    
    # 提供 config 参数
    config = {"configurable": {"thread_id": "support_003"}}
    
    final_state = graph.invoke(initial_state, config)
    
    print("\n对话历史:")
    for msg in final_state["messages"]:
        role = "用户" if msg.type == "human" else "支持"
        print(f"\n[{role}]:")
        content = msg.content[:400] + "..." if len(msg.content) > 400 else msg.content
        print(content)
    
    print(f"\n最终处理 agent: {final_state['current_agent']}")


if __name__ == "__main__":
    # 运行所有测试
    test_simple_issue()
    test_medium_issue()
    test_complex_issue()


场景 1：简单问题 - 密码重置

对话历史:

[用户]:
我忘记了账户密码，该怎么办？

[支持]:
请您尝试以下步骤来重置您的账户密码：

1. 在登录页面，寻找“忘记密码？”或“重置密码”的链接。
2. 点击该链接，您可能需要输入您的电子邮件地址或用户名。
3. 系统会向您提供的电子邮件发送一封重置密码的邮件。请检查您的邮箱，包括垃圾邮件文件夹。
4. 按照邮件中的指示重置您的密码。

如果您没有收到重置邮件或者仍然无法访问账户，请告诉我，我将转移给高级技术团队处理。[ESCALATE_TO_TIER2]

[支持]:
请您尝试以下步骤来重置您的账户密码：

1. 在登录页面，寻找“忘记密码？”或“重置密码”的链接。
2. 点击该链接，您可能需要输入您的电子邮件地址或用户名。
3. 系统会向您提供的电子邮件发送一封重置密码的邮件。请检查您的邮箱，包括垃圾邮件文件夹。
4. 按照邮件中的指示重置您的密码。

如果您没有收到重置邮件或者仍然无法访问账户，请告诉我，我将转移给高级技术团队处理。[ESCALATE_TO_TIER2]

最终处理 agent: tier2

场景 2：中等问题 - 应用性能缓慢

对话历史:

[用户]:
我们的应用在处理 100 万用户并发时性能严重下降。这是应用架构的问题吗？

[支持]:
性能问题可能由多种因素引起，确实可能与应用架构有关，但也可能涉及其他方面，例如数据库性能、网络延迟、服务器资源等。以下是一些可能需要检查的方面：

1. **应用架构**：是否采用了可扩展的架构设计？比如微服务架构可能在处理大量请求时表现更好。

2. **数据库性能**：查询是否经过优化？是否有索引缺失或锁竞争？

3. **服务器资源**：CPU、内存、IO等资源是否足够？是否存在瓶颈？

4. **负载均衡**：是否合理分配请求到多个服务器？

5. **缓存机制**：是否使用了缓存来减少数据库的负担？

建议您从这些方面进行排查。如果问题复杂，可能需要更深入的分析，您可以考虑将问题转移...

[支持]:
性能问题可能由多种因素引起，确实可能与应用架构有关，但也可能涉及其他方面，例如数据库性能、网络延迟、服务器资源等。以下是一些可能需要检查的方面：

1. **应用架构**：是否采用了可扩展的架构设计？比如微服务架构可能在处理大量请求